In [1]:
import torch
torch.set_num_threads(16)
torch.set_num_interop_threads(16)

import os
os.environ["DGL_USE_GRAPHBOLT"] = "0"

import cornac
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.eval_methods.propensity_stratified_evaluation import PropensityStratifiedEvaluation
from cornac.models import MF, PMF, BPR, SANSA, BiVAECF, LightGCN, RecVAE, EASE, NGCF, VAECF, IBPR, NeuMF, HPF
from cornac.metrics import Precision, Recall, NDCG, MAP, MRR

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split




SEED = 123
metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP(), MRR()]
movielens_100k_dataset = cornac.datasets.movielens.load_feedback(variant="100k")

rs = RatioSplit(data=movielens_100k_dataset, test_size=0.2, rating_threshold=0.0, seed=SEED)
rs.train_set.csr_matrix, rs.test_set.csr_matrix


# Force use of GPU:0
m = LightGCN(verbose=False, num_epochs=10)
n = NGCF(verbose=False, num_epochs=10)


cornac.Experiment(
    eval_method=rs,
    models=[m, n],
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


/Users/a-shyraliev/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/a-shyraliev/Library/Python/3.9/lib/python/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()



TEST:
...
         |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
LightGCN | 0.1656 | 0.5085 |  0.2656 |       0.2206 |    0.1389 |   49.8267 |   0.4887
NGCF     | 0.1855 | 0.4282 |  0.2502 |       0.2297 |    0.1244 |  240.5667 |   0.6630



In [3]:
import tensorflow as tf
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


In [36]:
import os
os.environ["DGL_USE_GRAPHBOLT"] = "0"

import cornac
from cornac.eval_methods import RatioSplit, StratifiedSplit
from cornac.eval_methods.propensity_stratified_evaluation import PropensityStratifiedEvaluation
from cornac.models import MF, PMF, BPR, SANSA, BiVAECF, LightGCN, RecVAE, EASE, NGCF, VAECF, IBPR, NeuMF, HPF
from cornac.metrics import Precision, Recall, NDCG, MAP, MRR

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split




SEED = 123
metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP(), MRR()]
movielens_100k_dataset = cornac.datasets.movielens.load_feedback(variant="20M")

rs = RatioSplit(data=movielens_100k_dataset, test_size=0.2, rating_threshold=0.0, seed=SEED)
rs.train_set.csr_matrix, rs.test_set.csr_matrix


m = MF(k=100)


cornac.Experiment(
    eval_method=rs,
    models=[m],
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()



TEST:
...
   |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
MF | 0.0206 | 0.1648 |  0.0626 |       0.0533 |    0.0232 |   38.0523 | 441.4083



### 1. Import datasets

In [2]:
SEED = 123

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]

In [3]:
movielens_100k_dataset = cornac.datasets.movielens.load_feedback()
# movielens_20M_dataset = cornac.datasets.movielens.load_feedback(variant="20m")

(<943x1656 sparse matrix of type '<class 'numpy.float64'>'
 	with 80000 stored elements in Compressed Sparse Row format>,
 <943x1656 sparse matrix of type '<class 'numpy.float64'>'
 	with 19971 stored elements in Compressed Sparse Row format>)

In [ ]:
m = RecVAE(use_gpu=False)

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]
movielens_100k_dataset = cornac.datasets.movielens.load_feedback()

rs = RatioSplit(data=movielens_100k_dataset, test_size=0.2, rating_threshold=0.2, seed=SEED)
rs.train_set.csr_matrix, rs.test_set.csr_matrix

cornac.Experiment(
    eval_method=rs,
    models=[m],
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


NameError: name 'RecVAE' is not defined

In [5]:
# steam_dataset_sample = sample_data(steam_dataset, n_users=100000, user_col='user_id')
_, steam_dataset_sample_str = stratified_ranking_split(
    steam_dataset,
    entity_field='user_id',
    test_size=0.001,
    random_state=SEED,
)

rs = RatioSplit(data=steam_dataset_sample_str.values, test_size=0.2, rating_threshold=0.0, seed=SEED)
print(rs.train_set.csr_matrix.shape, rs.test_set.csr_matrix.shape)

cornac.Experiment(
    eval_method=rs,
    models=[RecVAE(use_gpu=False, seed=SEED)],
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

(12183, 5343) (12183, 5343)


: 

### 1. MF hyperparams optimization

In [25]:
mf_models = [
    MF(name='MF k=10, lambda_reg=0.02', k=10, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
    MF(name='MF k=50, lambda_reg=0.02', k=50, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
    MF(name='MF k=100, lambda_reg=0.02', k=100, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
    MF(name='MF k=10, lambda_reg=0.1', k=10, max_iter=200, lambda_reg=0.1, use_bias=True, seed=SEED),
    MF(name='MF k=50, lambda_reg=0.1', k=50, max_iter=200, lambda_reg=0.1, use_bias=True, seed=SEED),
    MF(name='MF k=100, lambda_reg=0.1', k=100, max_iter=200, lambda_reg=0.1, use_bias=True, seed=SEED),
    MF(name='MF k=10, lambda_reg=1', k=10, max_iter=200, lambda_reg=1, use_bias=True, seed=SEED),
    MF(name='MF k=50, lambda_reg=1', k=50, max_iter=200, lambda_reg=1, use_bias=True, seed=SEED),
    MF(name='MF k=100, lambda_reg=1', k=100, max_iter=200, lambda_reg=1, use_bias=True, seed=SEED),
]

In [ ]:
cornac.Experiment(
    eval_method=rs,
    models=mf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                          | MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------- + --- + ------- + ------------ + --------- + --------- + --------
MF k=10, lambda_reg=0.02  | nan |  0.0001 |       0.0000 |    0.0003 |    0.6121 |  21.6584
MF k=50, lambda_reg=0.02  | nan |  0.0001 |       0.0000 |    0.0003 |    1.4565 |  31.7773
MF k=100, lambda_reg=0.02 | nan |  0.0001 |       0.0000 |    0.0003 |    4.6947 |  30.5076
MF k=10, lambda_reg=0.1   | nan |  0.0001 |       0.0000 |    0.0003 |    0.5838 |  21.4168
MF k=50, lambda_reg=0.1   | nan |  0.0001 |       0.0000 |    0.0003 |    1.4895 |  32.6081
MF k=100, lambda_reg=0.1  | nan |  0.0001 |       0.0000 |    0.0003 |    5.0005 |  32.5445
MF k=10, lambda_reg=1     | nan |  0.0001 |       0.0000 |    0.0003 |    0.6227 |  21.9549
MF k=50, lambda_reg=1     | nan |  0.0001 |       0.0000 |    0.0003 |    1.6991 |  32.9640
MF k=100, lambda_reg=1    | nan |  0.0001 |       0.0000 |    0.0003 

### 2. PMF hyperparams optimization

In [27]:
pmf_models = [
    PMF(name='PMF k=5, lambda_reg=0.001', k=5, max_iter=200, lambda_reg=0.001, seed=SEED),
    PMF(name='PMF k=10, lambda_reg=0.001', k=10, max_iter=200, lambda_reg=0.001, seed=SEED),
    PMF(name='PMF k=15, lambda_reg=0.001', k=15, max_iter=200, lambda_reg=0.001, seed=SEED),
    PMF(name='PMF k=5, lambda_reg=0.1', k=5, max_iter=200, lambda_reg=0.1, seed=SEED),
    PMF(name='PMF k=10, lambda_reg=0.1', k=10, max_iter=200, lambda_reg=0.1, seed=SEED),
    PMF(name='PMF k=15, lambda_reg=0.1', k=15, max_iter=200, lambda_reg=0.1, seed=SEED),
    PMF(name='PMF k=5, lambda_reg=1', k=5, max_iter=200, lambda_reg=1, seed=SEED),
    PMF(name='PMF k=10, lambda_reg=1', k=10, max_iter=200, lambda_reg=1, seed=SEED),
    PMF(name='PMF k=15, lambda_reg=1', k=15, max_iter=200, lambda_reg=1, seed=SEED),
]

In [28]:
cornac.Experiment(
    eval_method=rs,
    models=pmf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                           |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
PMF k=5, lambda_reg=0.001  | 0.0019 |  0.0019 |       0.0005 |    0.0039 |    5.5292 |  44.1669
PMF k=10, lambda_reg=0.001 | 0.0017 |  0.0017 |       0.0005 |    0.0036 |    9.4754 |  47.0882
PMF k=15, lambda_reg=0.001 | 0.0008 |  0.0005 |       0.0001 |    0.0011 |   14.2352 |  45.9387
PMF k=5, lambda_reg=0.1    | 0.0007 |  0.0000 |       0.0000 |    0.0000 |    5.2613 |  45.3645
PMF k=10, lambda_reg=0.1   | 0.0007 |  0.0000 |       0.0000 |    0.0000 |   10.0980 |  47.8467
PMF k=15, lambda_reg=0.1   | 0.0007 |  0.0000 |       0.0000 |    0.0000 |   14.9663 |  47.6700
PMF k=5, lambda_reg=1      | 0.0006 |  0.0002 |       0.0001 |    0.0005 |    5.5415 |  45.5808
PMF k=10, lambda_reg=1     | 0.0008 |  0.0004 |       0.0001 |    0.0006 |   10.5913 |  47.4731
PMF k=15, lambda_reg=1     | 

### 3. BPR hyperparams optimization

In [29]:
bpr_models = [
    BPR(name='BPR k=10, lambda_reg=0.01', k=10, max_iter=200, lambda_reg=0.01, seed=SEED),
    BPR(name='BPR k=50, lambda_reg=0.01', k=50, max_iter=200, lambda_reg=0.01, seed=SEED),
    BPR(name='BPR k=100, lambda_reg=0.01', k=100, max_iter=200, lambda_reg=0.01, seed=SEED),
    BPR(name='BPR k=10, lambda_reg=0.1', k=10, max_iter=200, lambda_reg=0.1, seed=SEED),
    BPR(name='BPR k=50, lambda_reg=0.1', k=50, max_iter=200, lambda_reg=0.1, seed=SEED),
    BPR(name='BPR k=100, lambda_reg=0.1', k=100, max_iter=200, lambda_reg=0.1, seed=SEED),
    BPR(name='BPR k=10, lambda_reg=1', k=10, max_iter=200, lambda_reg=1, seed=SEED),
    BPR(name='BPR k=50, lambda_reg=1', k=50, max_iter=200, lambda_reg=1, seed=SEED),
    BPR(name='BPR k=100, lambda_reg=1', k=100, max_iter=200, lambda_reg=1, seed=SEED),
]

In [30]:
cornac.Experiment(
    eval_method=rs,
    models=bpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                           |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
BPR k=10, lambda_reg=0.01  | 0.0308 |  0.0312 |       0.0092 |    0.0561 |    3.9516 |  45.1212
BPR k=50, lambda_reg=0.01  | 0.0309 |  0.0316 |       0.0094 |    0.0575 |    8.2983 |  57.4187
BPR k=100, lambda_reg=0.01 | 0.0307 |  0.0313 |       0.0093 |    0.0566 |   11.7384 |  57.0230
BPR k=10, lambda_reg=0.1   | 0.0307 |  0.0310 |       0.0090 |    0.0554 |    4.2672 |  47.3254
BPR k=50, lambda_reg=0.1   | 0.0303 |  0.0308 |       0.0091 |    0.0565 |    8.4085 |  57.3794
BPR k=100, lambda_reg=0.1  | 0.0281 |  0.0283 |       0.0087 |    0.0544 |   12.1549 |  55.3509
BPR k=10, lambda_reg=1     | 0.0300 |  0.0307 |       0.0092 |    0.0560 |    3.9472 |  44.8775
BPR k=50, lambda_reg=1     | 0.0299 |  0.0289 |       0.0081 |    0.0509 |    7.5961 |  52.7927
BPR k=100, lambda_reg=1    | 

### 4. BiVAECF hyperparams optimization

In [11]:
bivaecf_models = [
    BiVAECF(name='BiVAECF k=10, encoder_structure=[20]', k=10, encoder_structure=[20], use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=20, encoder_structure=[20]', k=50, encoder_structure=[20], use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=30, encoder_structure=[20]', k=100, encoder_structure=[20], use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=10, encoder_structure=[40]', k=10, encoder_structure=[40], use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=50, encoder_structure=[40]', k=50, encoder_structure=[40], use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=100, encoder_structure=[40]', k=100, encoder_structure=[40], use_gpu=False, seed=SEED),
]

In [12]:
cornac.Experiment(
    eval_method=rs,
    models=bivaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

: 

### 5. RecVAE hyperparams optimization

In [8]:
recvae_models = [
    RecVAE(name='RecVAE hidden_dim=600, latent_dim=100', hidden_dim=600, latent_dim=100, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=600, latent_dim=200', hidden_dim=600, latent_dim=200, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=600, latent_dim=300', hidden_dim=600, latent_dim=300, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=800, latent_dim=100', hidden_dim=800, latent_dim=100, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=800, latent_dim=200', hidden_dim=800, latent_dim=200, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=800, latent_dim=300', hidden_dim=800, latent_dim=300, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=1000, latent_dim=100', hidden_dim=1000, latent_dim=100, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=1000, latent_dim=200', hidden_dim=1000, latent_dim=200, use_gpu=False, seed=SEED),
    RecVAE(name='RecVAE hidden_dim=1000, latent_dim=300', hidden_dim=1000, latent_dim=300, use_gpu=False, seed=SEED),
]

In [9]:
cornac.Experiment(
    eval_method=rs,
    models=recvae_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

: 

: 

In [ ]:
# (rs.train_set.csr_matrix.toarray() != 0).sum(1)

In [ ]:
rs = RatioSplit(data=steam_recommendations, test_size=0.2, rating_threshold=0.0, seed=123)

mf = MF(k=10, max_iter=200, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)
pmf = PMF(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123)
bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)

models = [mf, pmf, bpr]

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]

cornac.Experiment(
    eval_method=rs,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/cornac/data/dataset.py:335: UserWarning: 1 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)



TEST:
...
    |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------- + ------------ + --------- + --------- + --------
MF  |    nan |  0.0025 |       0.0008 |    0.0057 |   42.5482 | 175.5171
PMF | 0.0029 |  0.0004 |       0.0001 |    0.0010 |  294.0087 | 197.7616
BPR | 0.0654 |  0.0730 |       0.0191 |    0.1435 |  254.8315 | 199.4555



In [4]:
amzn_clothing = cornac.datasets.amazon_clothing.load_feedback()
rs = RatioSplit(data=movielens_100k_dataset, test_size=0.2, rating_threshold=0.0, seed=123)

mf = MF(k=10, max_iter=200, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)
pmf = PMF(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123)
bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)

models = [mf, pmf, bpr]

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]

cornac.Experiment(
    eval_method=rs,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

Data from https://static.preferred.ai/cornac/datasets/amazon_clothing/rating.zip
will be cached into /Users/a-shyraliev/.cornac/amazon_clothing/rating.txt


164kB [00:01, 111kB/s]                             


Unzipping ...
File cached!

TEST:
...
    |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------- + ------------ + --------- + --------- + --------
MF  | 0.0322 |  0.0256 |       0.0276 |    0.0131 |    0.1320 |   0.3189
PMF | 0.0510 |  0.0801 |       0.0724 |    0.0324 |    2.1066 |   0.2911
BPR | 0.1336 |  0.2166 |       0.1900 |    0.1154 |    1.0722 |   0.2967



In [ ]:
rs = RatioSplit(data=ml_20M, test_size=0.2, rating_threshold=0.0, seed=123)

### 2. Initialize models, here we are comparing: Biased MF, PMF, and BPR

In [6]:
# mf = MF(k=10, max_iter=200, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)
# pmf = PMF(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123)
# bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)
# sansa_cholmod = SANSA(
#     name="SANSA (CHOLMOD)",
#     l2=500.0,
#     weight_matrix_density=1e-2,
#     compute_gramian=True,
#     factorizer_class="CHOLMOD",
#     factorizer_shift_step=1e-3,
#     factorizer_shift_multiplier=2.0,
#     inverter_scans=5,
#     inverter_finetune_steps=20,
#     use_absolute_value_scores=False,
#     verbose=False,
# )

# sansa_icf = SANSA(
#     name="SANSA (ICF)",
#     l2=10.0,
#     weight_matrix_density=1e-2,
#     compute_gramian=True,
#     factorizer_class="ICF",
#     factorizer_shift_step=1e-3,
#     factorizer_shift_multiplier=2.0,
#     inverter_scans=5,
#     inverter_finetune_steps=20,
#     use_absolute_value_scores=False,
#     verbose=False,
# )

bivaecf = BiVAECF(
    k=10,
    use_gpu=False,
)

lightgcn = LightGCN()

# recvae = RecVAE(
#     use_gpu=False,
# )

ease = EASE()

ngcf = NGCF()

# vaecf = VAECF(
#     use_gpu=False,
# )

ibpr = IBPR()

neumf = NeuMF()

hpf = HPF()

models = [bivaecf, ease, ibpr, neumf, hpf]# lightgcn, ngcf] sansa_cholmod, sansa_icf, mf, pmf, bpr, recvae, vaecf

### 3. Evaluation metrics

In [7]:
metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]

In [8]:
cornac.Experiment(
    eval_method=rs,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

: 

In [3]:
ml_100k = cornac.datasets.movielens.load_feedback()


In [ ]:
models = [sansa_icf, mf, pmf, bpr, bivaecf, recvae, ease, vaecf, ibpr, neumf, hpf]

cornac.Experiment(
    eval_method=rs_mod,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

INFO:sansa.core.factorizers:Computing incomplete Cholesky decomposition of X^TX + 10.0*I...
INFO:sansa.core.factorizers:Finding a fill-in reducing ordering (method = colamd)...
INFO:sansa.core.factorizers:Computing X^TX...
INFO:sansa.core.factorizers:
                X^TX info:
                    shape = (1656, 1656) 
                    nnz = 1722862 
                    density = 62.824614% 
                    size = 13.8 MB
                
                Attempting incomplete factorization of a relatively dense matrix (62.824614% dense). 
                This is unstable:
                 - the factorization might fail and automatically restart with additional regularization
                 - the resulting approximate factor might be of lesser quality
                You may want to try CHOLMODGramianFactorizer instead of ICFGramianFactorizer 
                (requires more memory but is likely faster and more accurate).
                
INFO:sansa.core.factorizers:Sorting indi

In [2]:
import torch
torch.__version__

'2.2.0'

In [1]:
import dgl
dgl.__version__


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 7

ModuleNotFoundError: No module named 'torch.utils._import_utils'

In [87]:
ml_20M = cornac.datasets.movielens.load_feedback(variant="20m")
rs = RatioSplit(data=ml_20M, test_size=0.2, rating_threshold=0.0, seed=123)

mf = MF(k=10, max_iter=200, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)
pmf = PMF(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123)
bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)

models = [mf, pmf, bpr]

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP()]

cornac.Experiment(
    eval_method=rs,
    models=models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
    |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------- + ------------ + --------- + --------- + --------
MF  | 0.0031 |  0.0003 |       0.0004 |    0.0002 |   31.1900 | 327.8577
PMF | 0.0108 |  0.0191 |       0.0197 |    0.0099 |  515.0184 | 330.4225
BPR | 0.1050 |  0.1940 |       0.1690 |    0.0777 |  577.5718 | 315.8166

